In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import random

In [15]:
ids_with_genres = pd.read_csv("genres_with_ids.csv", sep=",")
ids_with_themes = pd.read_csv("themes_with_ids.csv", sep=",")

In [16]:
df = pd.read_json('input_data_example.json')
df

,id,genres,name,themes
0,121,"[{'id': 31, 'name': 'Adventure'}, {'id': 33, '...",Minecraft,"[{'id': 17, 'name': 'Fantasy'}, {'id': 21, 'na..."
1,1879,"[{'id': 8, 'name': 'Platform'}, {'id': 12, 'na...",Terraria,"[{'id': 1, 'name': 'Action'}, {'id': 18, 'name..."
2,8173,"[{'id': 5, 'name': 'Shooter'}, {'id': 15, 'nam...",Overwatch,"[{'id': 1, 'name': 'Action'}, {'id': 18, 'name..."
3,9066,"[{'id': 13, 'name': 'Simulator'}, {'id': 15, '...",Cities: Skylines,"[{'id': 28, 'name': 'Business'}, {'id': 33, 'n..."
4,9527,"[{'id': 8, 'name': 'Platform'}, {'id': 10, 'na...",SpeedRunners,"[{'id': 1, 'name': 'Action'}, {'id': 40, 'name..."
5,135400,"[{'id': 13, 'name': 'Simulator'}, {'id': 31, '...",Minecraft,"[{'id': 17, 'name': 'Fantasy'}, {'id': 21, 'na..."


In [4]:
genre_list = []
theme_list = []
Dict = {}
# for each game in the database
for i, game in df.iterrows():
    
    # add each genre of that game to a list
    for j, genre in enumerate(game.genres):
        add_genre = game.genres[j]['id']
        genre_list.append(add_genre)

    # add each theme of that game to a list
    for jj, theme in enumerate(game.themes):
        add_theme = game.themes[jj]['id']
        theme_list.append(add_theme)
    
    # create a dictionary of names, ids, and the genre and theme lists    
    Dict[i] = {'game_name': df['name'][i], 'game_id':df['id'][i],'genre_list':genre_list, 'theme_list':theme_list}
    # empty the to be added lists for the next game
    genre_list = []
    theme_list = []

# convert dict to df    
ggt_df = pd.DataFrame.from_dict(Dict,orient='index')
ggt_df

,game_name,game_id,genre_list,theme_list
0,Minecraft,121,"[31, 33]","[17, 21, 33, 38, 41]"
1,Terraria,1879,"[8, 12, 13, 15, 31, 32]","[1, 18, 19, 21, 33, 38]"
2,Overwatch,8173,"[5, 15]","[1, 18, 27]"
3,Cities: Skylines,9066,"[13, 15]","[28, 33]"
4,SpeedRunners,9527,"[8, 10, 14, 32, 33]","[1, 40]"
5,Minecraft,135400,"[13, 31, 33]","[17, 21, 33, 38, 41]"


In [5]:
# get list of all themes
theme_columns = list(set([j for i in ggt_df['theme_list'].tolist() for j in i]))

# copy over name and id columns to new df
selected_columns = ggt_df[["game_name","game_id"]]
choice_theme = selected_columns.copy()

In [6]:
# Iterating over every list to create and fill values into columns for the game database
for j in theme_columns:
    choice_theme[j] = 0
for i in range(ggt_df.shape[0]):
    for j in theme_columns:
        if(j in ggt_df['theme_list'].iloc[i]):
            choice_theme.loc[i,j] = 1
            choice_theme.fillna(0, inplace=True)
choice_theme

,game_name,game_id,33,1,38,40,41,17,18,19,21,27,28
0,Minecraft,121,1,0,1,0,1,1,0,0,1,0,0
1,Terraria,1879,1,1,1,0,0,0,1,1,1,0,0
2,Overwatch,8173,0,1,0,0,0,0,1,0,0,1,0
3,Cities: Skylines,9066,1,0,0,0,0,0,0,0,0,0,1
4,SpeedRunners,9527,0,1,0,1,0,0,0,0,0,0,0
5,Minecraft,135400,1,0,1,0,1,1,0,0,1,0,0


In [7]:
# extracting info
features = theme_columns

X_train_dict = choice_theme[features].to_dict(orient="records")
y_train = choice_theme["game_name"]

In [8]:
# extracting info
X_train = choice_theme[theme_columns]
y_train = choice_theme["game_name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [10]:
# simulate user selecting a game
choice = ggt_df.loc[ggt_df['game_id'] == choice_theme.iloc[random.randint(0,5)].game_id]
print("User chose game " + choice['game_name'].to_string())


# create a new df where we will put info from the user's input
input_df = pd.DataFrame(columns=theme_columns, index=[0])


# Iterating over every list to create and fill values into columns for user input
for j in theme_columns:
    input_df[j] = 0
for i in range(choice.shape[0]):
    for j in theme_columns:
        if(j in choice['theme_list'].iloc[i]):
            input_df.loc[i,j] = 1

# store this in x_new
x_new = input_df.transpose().squeeze()

User chose game 1    Terraria


In [11]:
# Features in our model
features = theme_columns

X_train_dict = choice_theme[features].to_dict(orient="records")
y_train = choice_theme["game_name"]

In [12]:
X_train = choice_theme[theme_columns]
y_train = choice_theme["game_name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [13]:
x_new_sc = (x_new - X_train.mean()) / X_train.std()
x_new_sc

33    0.645497
1     0.912871
38    0.912871
40   -0.408248
41   -0.645497
17   -0.645497
18    1.290994
19    2.041241
21    0.912871
27   -0.408248
28   -0.408248
dtype: float64

In [14]:
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()

inds_nearest = dists_sorted.index[:9]

choice_theme.loc[inds_nearest]

,game_name,game_id,33,1,38,40,41,17,18,19,21,27,28
1,Terraria,1879,1,1,1,0,0,0,1,1,1,0,0
0,Minecraft,121,1,0,1,0,1,1,0,0,1,0,0
5,Minecraft,135400,1,0,1,0,1,1,0,0,1,0,0
2,Overwatch,8173,0,1,0,0,0,0,1,0,0,1,0
3,Cities: Skylines,9066,1,0,0,0,0,0,0,0,0,0,1
4,SpeedRunners,9527,0,1,0,1,0,0,0,0,0,0,0
